## Imports

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio as ipy_audio

from quicktranscribe import tonic, pitch, wave, kde

- Clone and install https://github.com/MTG/pycompmusic
- Download data using `download-dunya.py`

In [ ]:
DATA_DIR = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"

## Plot Annotations

In [ ]:
def plot_annotations(track):
    ctonic = tonic.read_tonic(DATA_DIR + track + ".ctonic.txt")
    pitch_annotations, aps = pitch.read_pitch(DATA_DIR + track + ".pitch.txt")
    
    pv = pitch.PitchValidator(audio_array=np.array([]), sampling_rate=0)
    pv.set_annotation(pitch_annotations=pitch_annotations, annotation_rate=aps)
    pv.set_tonic(ctonic)
    
    pv.plot_annotations_hist()

## Compare

In [ ]:
plot_annotations("Omkar Dadarkar - Raag Todi")

In [ ]:
plot_annotations("Omkar Dadarkar - Raag Multani")

In [ ]:
plot_annotations("Kaustuv Kanti Ganguli - Raag Puriya Dhanashree")

In [ ]:
plot_annotations("Kaustuv Kanti Ganguli - Raag Shree")

## Plot Detections

In [ ]:
def read_sample_and_tonic(track_path):
    
    ctonic = tonic.read_tonic(track_path + ".ctonic.txt")
    # metadata = tonic.read_metadata(track_path + ".json")
    # pitch_annotations, aps = pitch.read_pitch(track_path + ".pitch.txt")
    
    # # full audio
    # y_sample, sr = wave.get_audio(track_path + ".mp3")

    # # 10-minute sample
    start=10*60
    end=20*60
    y_stereo, sr = wave.read_audio_section(track_path + ".mp3", start, end)
    y_sample = librosa.to_mono(y_stereo.T)
    # ipy_audio(data=y_sample, rate=sr)
    
    return y_sample, sr, ctonic

In [ ]:
for track_mp3 in glob.glob(DATA_DIR + "*Bhoop*.mp3"):
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="green")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

In [ ]:
todis = {
    # "Ajoy Chakrabarty - Todi": None,  # corrupted
    "Kumar Gandharva - Raag Todi": None,
    "Omkar Dadarkar - Raag Todi": None,
}
multanis = {
    "Ajoy Chakrabarty - Multani": None,
    "Omkar Dadarkar - Raag Multani": None,
}

In [ ]:
for track_name in todis:
    track_mp3 = glob.glob(DATA_DIR + f"*{track_name}*mp3")[0]
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)
    todis[track_name] = kde_sample
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
    plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

plt.show()

In [ ]:
for track_name in multanis:
    track_mp3 = glob.glob(DATA_DIR + f"*{track_name}*mp3")[0]
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)
    multanis[track_name] = kde_sample
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="olivedrab")
    plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

plt.show()

In [ ]:
malkaunses = {
    "Satyasheel Deshpande - Raag Malkauns": None,
    # "Ajoy Chakrabarty - Malkauns": None,
}
bhoops = {
    "Omkar Dadarkar - Raag Bhoopali": None,
    "Sameehan Kashalkar - Raag Bhoopali": None,
}
meghs = {
    "Anol Chatterjee - Raag Megh": None,
}

In [ ]:
for track_name in malkaunses:
    track_mp3 = glob.glob(DATA_DIR + f"*{track_name}*mp3")[0]
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)
    malkaunses[track_name] = kde_sample
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="darkviolet")
    plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

plt.show()

In [ ]:
for track_name in bhoops:
    track_mp3 = glob.glob(DATA_DIR + f"*{track_name}*mp3")[0]
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)
    bhoops[track_name] = kde_sample
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="crimson")
    plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

plt.show()

In [ ]:
for track_name in meghs:
    track_mp3 = glob.glob(DATA_DIR + f"*{track_name}*mp3")[0]
    track_path = track_mp3[:-4]
    
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)
    meghs[track_name] = kde_sample
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="darkslateblue")
    plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    plt.xlabel("relative note index")
    plt.ylabel("normalized duration")

plt.show()

In [ ]:
malkauns = [0, 62, 98, 160, 200, 240]
bhoop = [0, 40, 78, 139, 178, 240]
megh = [0, 41, 100, 140, 202, 240]

In [ ]:
print(np.diff(malkauns))
print(np.diff(bhoop))
print(np.diff(megh))

In [ ]:
# out of 240, a syntonic comma will manifest as a delta of
(librosa.hz_to_midi(220*81/80) - librosa.hz_to_midi(220)) * 20